In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
state = AppState()
config = {}
trading_historical_data = {}
account = None
account = TradingAccount(app)
state.last_action = "SELL"
dataframe_dict = []
pd.set_option('display.max_columns', None)

In [28]:
def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])

    action = '' 
#     print(df_last['open'], df_last['close'], df_last['high'], df_last['low'])
    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True \
            and (ema12gtema26 is True or macdgtsignal is True) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':
        
        action = 'BUY'

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    if len(df) == 0:
        return df
    return df.iloc[iterations - 1:iterations]


In [29]:
def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [30]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    ta = TechnicalAnalysis(trading_dataCopy)
    ta.addAll()
    df = ta.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            # technical indicators
            ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
            ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
            goldencross = bool(df_last['goldencross'].values[0])
            macdgtsignal = bool(df_last['macdgtsignal'].values[0])
            macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
            ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
            ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
            macdltsignal = bool(df_last['macdltsignal'].values[0])
            macdltsignalco = bool(df_last['macdltsignalco'].values[0])
            obv = float(df_last['obv'].values[0])
            obv_pc = float(df_last['obv_pc'].values[0])
            elder_ray_buy = bool(df_last['eri_buy'].values[0])
            elder_ray_sell = bool(df_last['eri_sell'].values[0])
            rri = df_last['rri'].values[0]
            last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
            last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
            last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
            last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
            rri_buy = df_last['rri_buy'].values[0]
            rri_sell = df_last['rri_sell'].values[0]
            # if simulation interations < 200 set goldencross to true
            if app.isSimulation() and state.iterations < 200:
                goldencross = True

            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append([state.action, app.getGranularity(), price, ema12gtema26, ema12gtema26co,
                                     goldencross, macdgtsignal, macdgtsignalco, obv, obv_pc, elder_ray_buy,
                                     elder_ray_sell, rri, rri_buy, rri_sell, last_3_rri_buy, last_5_rri_buy, last_3_rri_sell, last_5_rri_sell, one_hr_high, one_hr_low, three_hr_high, three_hr_low, 
                                     three_six_high,three_six_low, six_12_high, six_12_low, six_hr_high, six_hr_low,
                                     nine_hr_high, nine_hr_low, twelve_hr_high, twelve_hr_low])
    

In [31]:
%%time
app = PyCryptoBot()
state = AppState()
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
                          'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
                          'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
                          'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
                          'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
                          'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
                          'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
                          'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
                          'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
                          'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
                          'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
                          'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
                          'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
                          'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
                       'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
                       'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
                       'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
                       'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
                       'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
                       'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
                       'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
                       'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
                       'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
                       'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
                       'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
                       'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
                       'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
                       'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
                       'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
                       'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
                       'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
                       'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
                       'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
                       'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
                       'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
                       'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
                       'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
                       'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
                        'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
                       'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
                       'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
                        'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
                       'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
                       'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
                       'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
                       'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
                       'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
                       'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
                       'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
                       'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
                       'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
                       'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
                       'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
                       'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
                       'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
                       'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
                       'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
                       'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
                       'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
                       'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
                       'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
                       'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
                       'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
                       'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
                       'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
                       'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
                       'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
                       'MDXBUSD', 'MASKBUSD']
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict,
                         columns=['action', 'Granularity', 'Price', 'Fast EMA gt', 'EMA CO',
                                  'Golden Cross', 'Macdgtsignal', 'Macdgtsignalco', 'Obv', 'Obv_pc',
                                  'Elder_ray_buy', 'elder_ray_sell', 'rri', 'rri_buy', 'rri_sell', 'last_3_rri_buy', 'last_5_rri_buy','last_3_rri_sell', 'last_5_rri_sell', 'one_hr_high', 'one_hr_low', 'three_hr_high',
                                  'three_hr_low', 'three_six_high','three_six_low', 'six_12_high', 'six_12_low',
                                  'six_hr_high', 'six_hr_low', 'nine_hr_high', 'nine_hr_low',
                                  'twelve_hr_high', 'twelve_hr_low'])


calculating  WTCBNB
calculating  BATBNB
calculating  NEOBNB
calculating  IOTABNB
calculating  XLMBNB
calculating  WABIBNB
calculating  LTCBNB
calculating  WAVESBNB
calculating  ICXBNB
calculating  BLZBNB
calculating  ZILBNB
calculating  ONTBNB
calculating  WANBNB
calculating  ADABNB
calculating  ZENBNB
calculating  EOSBNB
calculating  THETABNB
calculating  XRPBNB
calculating  ENJBNB
calculating  TRXBNB
calculating  ETCBNB
calculating  SCBNB
calculating  MFTBNB
calculating  VETBNB
calculating  RVNBNB
calculating  MITHBNB
calculating  BTTBNB
calculating  HOTBNB
calculating  FETBNB
calculating  XMRBNB
calculating  ZECBNB
calculating  IOSTBNB
calculating  CELRBNB
calculating  DASHBNB
calculating  MATICBNB
calculating  ATOMBNB
calculating  ONEBNB
calculating  FTMBNB
calculating  ALGOBNB
calculating  ANKRBNB
calculating  WINBNB
calculating  COSBNB
calculating  COCOSBNB
calculating  PERLBNB
calculating  CHZBNB
calculating  BANDBNB
calculating  XTZBNB
calculating  HBARBNB
calculating  STXBNB
c

calculating  ADABUSD
calculating  BCHBUSD
calculating  QTUMBUSD
calculating  VETBUSD
calculating  ICXBUSD
calculating  BNTBUSD
calculating  ATOMBUSD
calculating  DASHBUSD
calculating  NEOBUSD
calculating  WAVESBUSD
calculating  XTZBUSD
calculating  BATBUSD
calculating  ENJBUSD
calculating  NANOBUSD
calculating  ONTBUSD
calculating  RVNBUSD
calculating  ALGOBUSD
calculating  BTTBUSD
calculating  TOMOBUSD
calculating  XMRBUSD
calculating  ZECBUSD
calculating  DATABUSD
calculating  SOLBUSD
calculating  CTSIBUSD
calculating  HBARBUSD
calculating  MATICBUSD
calculating  WRXBUSD
calculating  ZILBUSD
calculating  KNCBUSD
calculating  LRCBUSD
calculating  IQBUSD
calculating  GBPBUSD
calculating  DGBBUSD
calculating  COMPBUSD
calculating  SXPBUSD
calculating  SNXBUSD
calculating  MKRBUSD
calculating  RUNEBUSD
calculating  MANABUSD
calculating  DOGEBUSD
calculating  ZRXBUSD
calculating  FIOBUSD
calculating  AVABUSD
calculating  IOTABUSD
calculating  BALBUSD
calculating  YFIBUSD
calculating  SRMB

In [32]:
buy_dataframe.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,11957.0,11957.000000,1.195700e+04,11957.000000,11950.000000,11894.000000,11894.000000,11894.000000,11894.000000,11702.000000,11702.000000,11598.000000,11598.000000,11894.000000,11894.000000,11894.000000,11894.000000,11894.000000,11894.000000
mean,3600.0,334.461346,2.435606e+07,5.150983,-0.363137,1.712600,-1.298179,2.805650,-2.285430,4.030702,-3.297547,5.414705,-4.696827,4.017479,-3.242262,4.987916,-3.986576,5.831608,-4.552020
std,0.0,3387.930568,6.277847e+08,416.960175,0.680942,2.584589,1.496785,3.772332,2.250659,6.860970,4.320843,10.435710,5.946696,5.214407,2.980064,6.582231,3.567780,9.026738,4.032037
min,3600.0,0.000101,-1.836598e+10,-14789.870000,-1.000000,-1.750000,-21.090000,-1.750000,-21.090000,-12.690000,-24.970000,-13.370000,-35.820000,-1.160000,-21.090000,-0.890000,-24.970000,-0.890000,-35.820000
25%,3600.0,0.079340,-7.025900e+02,-8.580000,-0.875000,0.480000,-1.770000,0.840000,-3.147500,0.540000,-5.560000,0.510000,-8.040000,1.210000,-4.440000,1.430000,-5.690000,1.630000,-6.400000
50%,3600.0,0.918600,1.994389e+05,-1.020000,-0.782609,1.100000,-0.920000,1.810000,-1.660000,2.660000,-2.575000,3.400000,-3.910000,2.700000,-2.340000,3.250000,-2.920000,3.670000,-3.340000
75%,3600.0,7.380500,3.391684e+06,6.770000,0.363289,2.160000,-0.340000,3.530000,-0.730000,5.760000,-0.660000,7.750000,-0.940000,4.990000,-1.160000,6.170000,-1.380000,7.030000,-1.600000
max,3600.0,50300.000000,2.896565e+10,24569.320000,1.000000,140.260000,1.870000,140.260000,1.870000,185.200000,21.970000,397.840000,38.530000,140.260000,1.860000,185.200000,1.860000,397.840000,1.860000


### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour



In [66]:
rri_buys = buy_dataframe.query("rri_buy==True").query("(`Fast EMA gt`==True&Macdgtsignal==True)")
rri_buys.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,2767.0,2767.000000,2.767000e+03,2767.00000,2767.000000,2730.000000,2730.000000,2730.00000,2730.000000,2708.000000,2708.000000,2673.000000,2673.000000,2730.000000,2730.000000,2730.000000,2730.000000,2730.000000,2730.000000
mean,3600.0,359.421529,2.270726e+07,-3.73176,-0.859314,1.645190,-1.154366,2.81563,-2.039524,4.217426,-2.839941,5.734201,-4.797692,4.062410,-2.889938,4.982000,-3.517070,5.991930,-4.165842
std,0.0,3623.453113,4.268316e+08,302.21427,0.077690,2.154952,1.392990,3.56720,1.932408,6.771333,3.903528,11.508613,5.668093,5.301788,2.657777,6.498644,3.160104,10.563652,3.677392
min,3600.0,0.000103,-1.126248e+10,-5968.57000,-1.000000,-1.000000,-21.090000,-0.89000,-21.090000,-12.690000,-20.280000,-12.690000,-35.340000,-0.890000,-21.090000,-0.890000,-21.090000,-0.310000,-35.340000
25%,3600.0,0.080880,-3.235390e+02,-13.06000,-0.919560,0.520000,-1.570000,0.88000,-2.810000,0.820000,-4.750000,1.020000,-7.480000,1.320000,-3.957500,1.532500,-4.890000,1.890000,-5.750000
50%,3600.0,0.902500,1.940714e+05,-3.99000,-0.846154,1.080000,-0.880000,1.81000,-1.620000,2.820000,-2.360000,3.810000,-4.280000,2.790000,-2.150000,3.260000,-2.690000,3.840000,-3.220000
75%,3600.0,7.365000,3.303818e+06,-0.00500,-0.791712,2.010000,-0.360000,3.58000,-0.702500,5.775000,-0.530000,7.570000,-1.480000,4.987500,-1.080000,5.990000,-1.240000,6.837500,-1.612500
max,3600.0,46406.100000,1.506829e+10,9715.72000,-0.750000,37.080000,1.750000,59.50000,1.750000,105.530000,21.970000,397.840000,34.460000,105.530000,1.640000,105.530000,1.270000,397.840000,1.270000


In [44]:
non_rri_buys = buy_dataframe.query("rri_buy==False")
non_rri_buys.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,4623.0,4623.000000,4.623000e+03,4623.000000,4616.000000,4612.000000,4612.000000,4612.000000,4612.000000,4464.000000,4464.000000,4415.000000,4415.000000,4612.000000,4612.000000,4612.000000,4612.000000,4612.000000,4612.000000
mean,3600.0,377.887056,2.834801e+07,18.719091,0.431908,1.744258,-1.517533,2.930941,-2.723818,4.152686,-3.479928,5.834750,-4.381185,4.222188,-3.595245,5.274892,-4.248133,6.085703,-4.604926
std,0.0,3565.684079,7.602306e+08,587.354925,0.400666,3.352942,1.582634,4.506943,2.469610,7.189431,4.488215,11.254273,5.944452,5.710130,3.066406,6.953744,3.575266,9.581612,3.888157
min,3600.0,0.000104,-1.836598e+10,-14789.870000,-1.000000,-1.720000,-15.540000,-1.630000,-16.370000,-10.250000,-22.410000,-13.370000,-35.820000,-0.890000,-17.870000,-0.890000,-22.410000,-0.890000,-35.820000
25%,3600.0,0.097550,3.344885e+02,-2.890000,0.205907,0.390000,-2.040000,0.690000,-3.780000,0.180000,-5.952500,0.305000,-7.740000,1.030000,-5.102500,1.220000,-6.080000,1.370000,-6.500000
50%,3600.0,1.019300,3.146094e+05,4.080000,0.522570,0.990000,-1.060000,1.680000,-2.000000,2.670000,-2.910000,3.600000,-3.700000,2.550000,-2.740000,3.170000,-3.240000,3.605000,-3.530000
75%,3600.0,7.825500,4.878634e+06,21.150000,0.738126,1.980000,-0.480000,3.530000,-0.940000,6.400000,-0.820000,8.855000,-0.690000,5.380000,-1.300000,6.850000,-1.620000,7.790000,-1.780000
max,3600.0,50300.000000,2.896565e+10,24569.320000,1.000000,140.260000,1.690000,140.260000,1.690000,107.720000,20.880000,394.220000,24.800000,140.260000,1.690000,140.260000,1.690000,394.220000,1.690000


In [ ]:
loss_buys = buy_dataframe.query("three_hr_high<1.1").query("three_six_high<0.05").query("six_12_high<-5")
loss_buys.describe()

In [ ]:
loss_buys_with_sell= loss_buys[loss_buys['Obv_pc'] < -5]
loss_buys_with_sell.describe()

In [ ]:
df_last['rri']

In [ ]:
trading_historical_data

In [ ]:
df_last['rri']